In [1]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta
import os.path
from datetime import datetime
#from sqlalchemy.sql import text

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
for k in ['DEIS']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema DEIS exists, will not be created


In [5]:
now = dt.now()
delta=0
status=400
while status!=200:
    date=now- timedelta(days=delta)
    print('testing date:'+str(date))
    dia=date.strftime("%d")
    mes=date.strftime("%m")
    year=str(date.year)
    url='http://deis.minsal.cl/wp-content/uploads/'+year+'/'+mes+'/DEFUNCIONES_FUENTE_DEIS_2016_'+year+'_'+dia+mes+year+'.rar'
    print('url: '+url)
    path='/data/ETLcache/'
    file=url.split(mes+'/')[1]


    if os.path.isfile(path+file):
        print ("Date exists on archive")
        flag=False
        break
    else:
        req = requests.get(url)
        status=req.status_code
        print(status)
        if status==200:
            print('success')
            print ("Creating file "+path+file)
            open(path+file, 'wb').write(req.content)
            flag=True
        else:
            print('failure, trying previous date')
            delta+=1

testing date:2021-02-04 03:27:03.845120
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_04022021.rar
404
failure, trying previous date
testing date:2021-02-03 03:27:03.845120
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_03022021.rar
404
failure, trying previous date
testing date:2021-02-02 03:27:03.845120
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_02022021.rar
404
failure, trying previous date
testing date:2021-02-01 03:27:03.845120
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_01022021.rar
404
failure, trying previous date
testing date:2021-01-31 03:27:03.845120
url: http://deis.minsal.cl/wp-content/uploads/2021/01/DEFUNCIONES_FUENTE_DEIS_2016_2021_31012021.rar
404
failure, trying previous date
testing date:2021-01-30 03:27:03.845120
url: http://deis.minsal.cl/wp-content/uploads/2021/01/DEFUNCIONES_FUENTE_DEIS_2016_20

In [6]:
if not os.path.exists(path+file.rsplit('.rar')[0]):
    print('unpacking RAR file')
    os.mkdir(path+file.rsplit('.rar')[0])
    patoolib.extract_archive(path+file, outdir=path+file.rsplit('.rar')[0])
else:
    print(path+file+' already in archive, no unpacking')

/data/ETLcache/DEFUNCIONES_FUENTE_DEIS_2016_2021_28012021.rar already in archive, no unpacking


In [7]:
current_year = dt.now().year

In [8]:
DBlast_date=cnx.execute('SELECT max("FECHA_DEF") FROM "DEIS"."DEIS_'+str(current_year)+'";')
DBlast_date=DBlast_date.first()[0]

In [9]:
DBlast_date

datetime.datetime(2021, 1, 27, 0, 0)

In [10]:
file_last_date=file.rsplit('.rar')[0].rsplit('_',1)[1]
file_last_date=file_last_date[:2]+'-'+file_last_date[2:4]+'-'+file_last_date[4:]
file_last_date=datetime.strptime(file_last_date, '%d-%m-%Y')

In [11]:
file_last_date

datetime.datetime(2021, 1, 28, 0, 0)

In [12]:
db_flag=file_last_date>DBlast_date

In [13]:
db_flag

True

In [14]:
def csvs_to_sql(deis,headers):
    schema='DEIS'
    headers=headers['Nombre de la variable']
    #headers=headers[1:]
    deis.columns=headers
    deis['Metropolitana']=0
    deis.loc[deis['GLOSA_REG_RES']=='Metropolitana de Santiago','Metropolitana']=1
    deis['Metropolitana']=deis['Metropolitana'].replace({0:'Resto de Chile',1:'Región Metropolitana'})
    deis['FECHA_DEF']=pd.to_datetime(deis['FECHA_DEF'],format='%Y-%m-%d',errors='coerce')
    deis['FECHA_DEF']=deis['FECHA_DEF'].dt.to_pydatetime()
    deis['FECHA_STR']=deis['FECHA_DEF'].dt.strftime('%d-%m-%Y')
    deis['COVID']=deis['DIAG1'].replace({'U071':'Covid PCR+','U072':'Covid Sospechoso'})
    deis.loc[~deis['DIAG1'].isin(['U071','U072']),'COVID']=np.nan

    if not 'ANO_DEF' in deis.columns:
        print('creando columna: ANO_DEF')
        deis['ANO_DEF']=deis['FECHA_DEF'].dt.year
        
    for y in deis['ANO_DEF'].unique():
        name='DEIS_'+str(y).split('.')[0]
        df=deis[deis['ANO_DEF']==y]
        if y==current_year:
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
        else:
            try:
                print("creating table (not overwrite mode)"+name+' ,schema: '+schema)
                df.to_sql(name, schema=schema,con=cnx,if_exists='fail')
            except:
                print('table '+name+' ,schema: '+schema+' already exists')
                
            
    deis_covid=deis[deis['COVID'].notnull()]
    name='DEIS_COVID'
    print("creating table "+name+' ,schema: '+schema)
    deis_covid.to_sql(name, schema=schema,con=cnx,if_exists='replace')
    
    edad_fecha=deis.loc[(deis['ANO_DEF']==2020) & (deis['COVID'].notnull())]
    edad_fecha=edad_fecha.groupby(['FECHA_DEF','EDAD_CANT'])['COVID'].count().reset_index()
    edad_fecha=edad_fecha.set_index('FECHA_DEF').groupby('EDAD_CANT').rolling('7d',min_periods=1)['COVID'].mean()
    edad_fecha=edad_fecha.reset_index()
    name='DEIS_EDAD_FECHA_ROLLING'
    print("creating table "+name+' ,schema: '+schema)
    edad_fecha.to_sql(name, schema=schema,con=cnx,if_exists='replace')

In [15]:
if db_flag==True:
    print('Reading files')
    for f in os.listdir(path+file.rsplit('.rar')[0]):
        if '.csv' in f:
            deis=pd.read_csv(path+file.rsplit('.rar')[0]+'/'+f,sep=';',encoding='latin1',header=0)
        if '.xlsx' in f:
            headers=pd.read_excel(path+file.rsplit('.rar')[0]+'/'+f,header=3,index_col=1)
    csvs_to_sql(deis,headers)
else:
    print('Files already loaded into DBASE, not reading')

Reading files


<ipython-input-15-134013f5df97>:7: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  headers=pd.read_excel(path+file.rsplit('.rar')[0]+'/'+f,header=3,index_col=1)


creating table DEIS_2021 ,schema: DEIS
creating table (not overwrite mode)DEIS_2020 ,schema: DEIS
table DEIS_2020 ,schema: DEIS already exists
creating table (not overwrite mode)DEIS_2019 ,schema: DEIS
table DEIS_2019 ,schema: DEIS already exists
creating table (not overwrite mode)DEIS_2018 ,schema: DEIS
table DEIS_2018 ,schema: DEIS already exists
creating table (not overwrite mode)DEIS_2017 ,schema: DEIS
table DEIS_2017 ,schema: DEIS already exists
creating table (not overwrite mode)DEIS_2016 ,schema: DEIS
table DEIS_2016 ,schema: DEIS already exists
creating table DEIS_COVID ,schema: DEIS
creating table DEIS_EDAD_FECHA_ROLLING ,schema: DEIS


In [16]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_DEIS.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/DEIS.ipynb')

0